In [8]:
from pandas import read_csv
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


In [9]:
def get_train_test(url, split_percent = 0.8):
    df = read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    n = len(data)
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]  
    return train_data, test_data, data

In [15]:
def get_XY(dat, time_steps):
    Y_ind = np.arange(time_steps, len(dat),time_steps)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps))
    return X, Y

In [16]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(dense_units = dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [17]:
def print_error(trainY, testY, train_Predict, test_Predict):
    train_rmse = math.sqrt(mean_squared_error(trainY, train_Predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_Predict))
    print('Train RMSE: %.2f' % (train_rmse))
    print('Test RMSE: %.2f' % (test_rmse))   

In [18]:
def plot_result(trainY, testY, train_predict, test_predict):
    actual = np.append(trainY, testY)
    predictions = np.append(train_predict, test_predict)
    rows = len(actual)
    plt.figure(figsize=(15,6),dpi=80)
    plt.plot(range(rows), actual, color='b', label='Actual')
    plt.plot(range(rows), predictions, color='r', label='Predictions')
    plt.axvline(x=len(trainY), color='g', linestyle='--')
    plt.legend(['Actual', 'Predictions'])
    plt.xlabel('Observation number after given time steps') 
    plt.ylabel('Sunspots scaled')
    plt.title('Actual vs Predicted Sunspots')

In [19]:
sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
timesteps =12
train_data, test_data, data = get_train_test(sunspots_url)
trainX, trainY = get_XY(train_data, timesteps)
testX, testY = get_XY(test_data, timesteps)


In [20]:
model = create_RNN(hidden_units=3, dense_units=1, input_shape=(timesteps,1), activation=['tanh','tanh'])
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

TypeError: Parameter to MergeFrom() must be instance of same class: expected tensorflow.TensorShapeProto got tensorflow.TensorShapeProto.

In [ ]:
#make predictions
train_predict = model.predict(trainX)
test_predict = model.predict(testX)

#print error
print_error(trainY, testY, train_predict, test_predict)

#plot result
plot_result(trainY, testY, train_predict, test_predict)